In [1]:
import platform
import os
import glob
import fnmatch
import pandas
from tqdm import notebook

In [2]:
platform.platform()

'Windows-7-6.1.7601-SP1'

All the data is on `\\resstore.unibe.ch\ana_rs_myct\`

In [3]:
if 'LINUX' in platform.platform():
    # LINUX
    ResStore = os.path.join(os.sep, 'home', 'habi', 'resstorage')
elif 'Windows' in platform.platform():
    ResStore = os.path.join('R://')
elif 'Darwin' in platform.platform():
    ResStore = os.path.join('/Volumes/ana_rs_myct', 'data')
print('We are loading all the data from %s' % ResStore)

We are loading all the data from R://


In [4]:
if os.path.isfile('alllogfiles.txt'):
    # Read previously saved logfiles from disk
    print('Reading previously saved file with a list of all found logfiles')
    with open('alllogfiles.txt', 'r') as f:
        # https://stackoverflow.com/a/15233379/323100
        alllogfiles = f.read().splitlines()
else:
    # Get *All* logfiles from disk
    # https://stackoverflow.com/a/14798263/323100
    print('Generating a list of all found logfiles found on the research storage')
    # This takes at least half an hour..
    import datetime
    print('Started looking at %s' % datetime.datetime.now())
    alllogfiles = glob.glob(os.path.join(ResStore, 'Archiv_Tape', '*', '**', '*.log'), recursive=True)
    print('Finished looking at %s' % datetime.datetime.now())
    # Save to disk for next use
    with open('alllogfiles.txt', "w") as output:
        for row in alllogfiles:
            output.write(str(row) + '\n')

Generating a list of all found logfiles found on the research storage
Started looking at 2020-04-07 20:56:03.846262
Finished looking at 2020-04-07 21:19:47.756862


In [5]:
#for l in alllogfiles:
#    print(l)

In [6]:
# len(alllogfiles)

In [7]:
logfiles = alllogfiles.copy()

In [8]:
print('We found %s logfiles' % len(logfiles))
# Remove all the ones that we don't want
# Do NOT do this in a loop, otherwise this does not work correctly
findstring = ['*_rec.log', '*_rectmp.log', '*_rec_???_.log', '*_rec_???.log', '*_rec_???_voi_.log',
              '*voi__voi_.log',  '_voi_.*.log', '*.ctan.log', '*.batman.log',
              '*\\STEPanizerizer.log', '*\\settings.log', '*\\example.log',
              '*\\rec\\*', '*\\VOI\\*']
for what in findstring:
    # Use the fnmatch library to filter out unnneded log files
    logfiles = [l for l in logfiles if not fnmatch.fnmatch(l, what)]
print('After removing all')
for what in findstring:
    print("\t- '%s'" % what)
print('files, we have %s logfiles left to work with' % len(logfiles))

We found 5035 logfiles
After removing all
	- '*_rec.log'
	- '*_rectmp.log'
	- '*_rec_???_.log'
	- '*_rec_???.log'
	- '*_rec_???_voi_.log'
	- '*voi__voi_.log'
	- '_voi_.*.log'
	- '*.ctan.log'
	- '*.batman.log'
	- '*\STEPanizerizer.log'
	- '*\settings.log'
	- '*\example.log'
	- '*\rec\*'
	- '*\VOI\*'
files, we have 3051 logfiles left to work with


In [9]:
# Check if there are some 'rec' left...
for f in logfiles:
    if 'rec' in f:
        print(f)

R://Archiv_Tape\Zebra-Fish_Ines\OrganVolume\Fishes\REC02\proj\rec02~00.log
R://Archiv_Tape\Zebra-Fish_Ines\OrganVolume\Fishes\REC02\proj\rec02.log
R://Archiv_Tape\Zebra-Fish_Ines\OrganVolume\Fishes\REC02\proj\rec02~02.log
R://Archiv_Tape\Zebra-Fish_Ines\OrganVolume\Fishes\REC02\proj\rec02~01.log
R://Archiv_Tape\Zebra-Fish_Ines\OrganVolume\Fishes\REC03\proj\rec03.log
R://Archiv_Tape\Zebra-Fish_Ines\OrganVolume\Fishes\REC03\proj\rec03~01.log
R://Archiv_Tape\Zebra-Fish_Ines\OrganVolume\Fishes\REC03\proj\rec03~00.log
R://Archiv_Tape\Zebra-Fish_Ines\OrganVolume\Fishes\REC03\proj\rec03~02.log
R://Archiv_Tape\Zebra-Fish_Ines\OrganVolume\Fishes\REC01\proj\rec01~01.log
R://Archiv_Tape\Zebra-Fish_Ines\OrganVolume\Fishes\REC01\proj\rec01~00.log
R://Archiv_Tape\Zebra-Fish_Ines\OrganVolume\Fishes\REC01\proj\rec01~02.log
R://Archiv_Tape\Zebra-Fish_Ines\OrganVolume\Fishes\REC01\proj\rec01.log
R://Archiv_Tape\Zebra-Fish_Ines\OrganVolume\Thorax\REC02\proj\rec02~00.log
R://Archiv_Tape\Zebra-Fish_Ines\Or

In [10]:
def get_numberofprojectionsfromlog(logfile, verbose=False):
    """Get the number of projections from the logfile"""
    with open(logfile, 'r') as f:
        if verbose:
            print('In %s we found that there should be' % logfile, end=' ')
        for line in f:
            # Sometimes Bruker writes 'of Files', sometimes 'Of Files'...
            if 'Number' and 'f Files' in line:
                numrec = int(line.split('=')[1])
        if verbose:
            print('%s projections' % numrec)
    try:
        return(numrec)
    except:
        print('No "number of files" found in %s' % logfile)
        return()

In [11]:
def get_numberofprojectionsfromdir(logfile, verbose=False):
    """
    Get the number of projections from the proj folder.
    Only look for TIFF files with 'prefix'+NUMERAL+.tif.
    Otherwise we also get the '_arc*.tif' and '_pp*.tif' files.
    We get the prefix from the logfile.
    """
    import re  # for regex searching
    if verbose:
        print('Based on %s' % logfile)
        print('Checking number of TIF files in %s' % os.path.dirname(logfile))
    with open(logfile, 'r') as f:
        prefix = []
        for line in f:
            # Get the prefix of the image files, which is handy for oversize scans
            if 'Filen' in line and 'fix' in line:
                prefix = line.split('=')[1].strip()
    if not prefix:
        print('No "prefix" found in %s' % logfile)
        return()
    if verbose:
        print('We are only looking for files with %sNUMERALS.tif' % prefix)  
    # ALL tiffiles, but only with with 'prefix' + numerals + .tif, hence exclding prefix_pp*.tif and prefix_ar*.tif
    # regex search based on https://stackoverflow.com/a/55810892/323100 and regexr.com/51lob
    tiffiles = [f for f in os.listdir(os.path.dirname(logfile)) if re.search(rf'{prefix}([0-9])*\.tif', f)]
    if verbose:
        print('And found %s such files' % len(tiffiles))
    try:
        return(len(tiffiles))
    except:
        print('Based on %s We were not able to find any tif files.' % logfile)
        return()

In [12]:
def get_listofprojections(logfile, verbose=False):
    """Get the actual projection names."""
    import re  # for regex searching
    if verbose:
        print('Based on %s' % logfile)
        print('Checking number of TIF files in %s' % os.path.dirname(logfile))
    with open(logfile, 'r') as f:
        prefix = []
        for line in f:
            # Get the prefix of the image files, which is handy for oversize scans
            if 'Filename' in line and 'fix' in line:
                prefix = line.split('=')[1].strip()
    if not prefix:
        print('No "prefix" found in %s' % logfile)
        return()
    if verbose:
        print('We are only looking for files with %sNUMERALS.tif' % prefix)            
    # ALL tiffiles, but only with with 'prefix' + numerals + .tif, hence exclding prefix_pp*.tif and prefix_ar*.tif
    # regex search based on https://stackoverflow.com/a/55810892/323100 and regexr.com/51lob
    tiffiles = [os.path.join(os.path.dirname(logfile), f) for f in os.listdir(os.path.dirname(logfile)) if re.search(rf'{prefix}([0-9])*\.tif', f)]
    if verbose:
        print('And found %s such files' % len(tiffiles))            
    try:
        return(tiffiles)
    except:
        print('Based on %s We were not able to find any tif files.' % logfile)
        return()

In [13]:
#for l in logfiles:
#    print(l)
#    print(get_numberofprojectionsfromdir(l),
#          get_numberofprojectionsfromlog(l))

In [14]:
# Make us a dataframe for saving all that we need
Data = pandas.DataFrame()

In [15]:
# Put the log files from above into the dataframe
Data['LogFile'] = logfiles

In [16]:
# Generate folder names
Data['Folder'] = [os.path.dirname(f) for f in Data['LogFile']]
Data['LastFolder'] = [os.path.basename(f) for f in Data['Folder']]

In [17]:
#for f in Data['LogFile']:
#    get_numberofprojectionsfromlog(f)

In [18]:
# TEMPORÄR
logfiles = [l for l in logfiles if not fnmatch.fnmatch(l, '*Ganz_Mumifiziert_18um_Cu0*')]

In [19]:
# Get the number of projections as stated in the directory
Data['NumProjFromLog'] = [get_numberofprojectionsfromlog(f) for f in Data['LogFile']]
#for f in logfiles:
#    get_numberofprojectionsfromlog(f, verbose=False)

In [20]:
# Get the number of projections that are present in the directory
Data['NumProjFromDirectory'] = [get_numberofprojectionsfromdir(f) for f in Data['LogFile']]
#for f in logfiles:
#    get_listofprojections(f, verbose=False)

In [21]:
def get_machine(logfile, verbose=False):
    """Get the machine name from the logfile"""
    with open(logfile, 'r') as f:
        if verbose:
            print('In %s we found that the scan was done on' % logfile, end=' ')
        for line in f:
            if 'Scanner=' in line:
                machine = str(line.split('=')[1]).strip()
        if verbose:
            print('the %s' % machine)
    try:
        return(machine)
    except:
        print('No "Scanner=" found in %s' % logfile)
        return()

In [22]:
# It seems that for (most of the) scans with the 1172 the NumProj in the log file is one more than the files on disk.
# Let's see if that is the case by extracting the machnie from the logfile and checking if NumProjDir+1 is the same as NumProjLog
Data['Machine'] = [get_machine(f) for f in Data['LogFile']]

In [252]:
def get_scandate(logfile, verbose=False):
    """Get the time and date of the scan from the logfile"""
    with open(logfile, 'r') as f:
        if verbose:
            print('In %s we found that the scan was done on' % logfile, end=' ')
        for line in f:
            if 'Study Date and Time' in line:
                try:
                    #print(line.split('=')[1].strip())
                    scandate = pandas.to_datetime(line.split('=')[1])
                except:
                    # Some time the time is written with HHh:MMm:SSs.
                    # We get rid of the denominators with some string splitting and replacing
                    #print(line.split('=')[1].strip('s\n').replace('m:',':').replace('h:',':'))
                    scandate = pandas.to_datetime(line.split('=')[1].strip('s\n').replace('m:',':').replace('h:',':'))
        if verbose:
            print('%s' % scandate)
    try:
        return(scandate)
    except:
        print('No "Study Date and Time=" found in %s' % logfile)
        return()

In [246]:
# Get the date of the scan
Data['ScanDate'] = [get_scandate(f) for f in Data['LogFile']]

In [23]:
# See if the number of projection in the log file is the same as the one on disk
Data['SameSame'] = [l == d for l,d in zip(Data['NumProjFromLog'], Data['NumProjFromDirectory'])]

In [254]:
set(Data.SameSame)

{False, True}

In [265]:
# Calculate the difference
Data['Difference'] = [abs(d-l) for d,l in zip(Data['NumProjFromDirectory'],Data['NumProjFromLog'])]

In [291]:
# Difference-check, the old way
# if Data['SameSame'].all():
#     print('All %s found logfiles say that we have as many TIF files as found in their respective folder' % len(Data))
# else:
#     Data['SameSamePlusMinusOne'] = ''
#     for c, row in Data.iterrows():
#         if row.SameSame:
#             Data.at[c,'SameSamePlusMinusOne'] = 'Sa'
#         else:
#             if row.NumProjFromDirectory + 1 == row.NumProjFromLog:
#                 # For the 1172, Bruker sometimes states in the logfile that there's a file more on disk than what we find
#                 # print(row.Machine)
#                 if '1172' in row.Machine:
#                     Data.at[c,'SameSamePlusMinusOne'] = 'Plus'
#                     pass
#                 else:
#                     print('Check %s. The Logfile says we should have %s files, we found %s files' % (row.LogFile,
#                                                                                                  row.NumProjFromLog,
#                                                                                                  row.NumProjFromDirectory))
#             elif row.NumProjFromDirectory - 1 == row.NumProjFromLog:
#                 # For the 1272, Bruker sometimes states in the logfile that there's a file less on disk than what we find
#                 # print(row.Machine)
#                 if '1272' in row.Machine:
#                     Data.at[c,'SameSamePlusMinusOne'] = 'Minus'
#                     pass
#                 else:
#                     print('Check %s. The Logfile says we should have %s files, we found %s files' % (row.LogFile,
#                                                                                                  row.NumProjFromLog,
#                                                                                                  row.NumProjFromDirectory))
#             elif row.NumProjFromDirectory == 0:
#                 pass
#                 #print('Check %s. We found %s files' % (row.LogFile, row.NumProjFromDirectory))
#             else:
#                 pass
#                 #print('Check %s. The Logfile says we should have %s files, we found %s files' % (row.LogFile,
#                 #                                                                                 row.NumProjFromLog,
#                 #                                                                                 row.NumProjFromDirectory))

Check if tif files are correct: http://www.blog.pythonlibrary.org/2020/02/09/how-to-check-if-a-file-is-a-valid-image-with-python/

In [27]:
def checkTIFfile(listoftiffs, subset=True, part=50, verbose=False):
    import imghdr
    import imageio
    import numpy
    howmany = len(listoftiffs)
    if not howmany:
        # There are no files
        return(False)
    if subset:
        # Get a random subset (part chooses it size) of the original, but at least one file (e.g. 1 + numpy.ceil)
        listoftiffs = numpy.random.choice(listoftiffs, size=int(1 + numpy.ceil(len(listoftiffs)/part)))
    allgood = [False] * len(listoftiffs)
    for c, f in notebook.tqdm(enumerate(listoftiffs), total=len(listoftiffs), desc='Checking %s of %s' % (len(listoftiffs), howmany)):
        img = imageio.imread(f)
        if verbose:
            print('%s is a %s file with size of %s and a mean of %0.2f' % (os.path.basename(f),
                                                                           imghdr.what(f),
                                                                           numpy.shape(img),
                                                                           numpy.mean(img)))
        # Consistencycheck: Is it a TIFF? AND Does it have a dimension? AND Is there some information in the image?
        if imghdr.what(f) == 'tiff' and  numpy.shape(img)[0] > 0 and numpy.mean(img) > 0:
            allgood[c] = True
    # Get unique values with list(set(our_list))
    return(list(listoftiffs), allgood)

In [28]:
#for c,row in Data.iterrows():
#    print('Checking %s' % row.Folder)
#    checkTIFfile(row.Projections,part=500)

In [29]:
# checkTIFfile(get_listofprojections(Data['LogFile'][3]), part=1111)

In [30]:
# Not as a list comprehension, but so that we get a progress bar
Data['CheckedTIFFs'] = None
Data['CheckedGood'] = None
for c, row in notebook.tqdm(Data.iterrows(),
                            total=len(Data),
                            desc='Checking TIFFs',):
    try:
        checkedlist, status = checkTIFfile(get_listofprojections(row['LogFile']), part=1111)
        Data.at[c,'CheckedTIFFs'] = checkedlist
        Data.at[c,'CheckedGood'] = status
    except:
        print('Cannot check %s, something is wrong' % row.Folder)

Cannot check R://Archiv_Tape\Neda\control01_lumbar_1_32um_50kv_nof\proj, something is wrong



Cannot check R://Archiv_Tape\Neda\Control01\lumbar\proj, something is wrong
Cannot check R://Archiv_Tape\Neda\Control01\lumbar\proj, something is wrong
Cannot check R://Archiv_Tape\Neda\Control01\lumbar\proj, something is wrong



Cannot check R://Archiv_Tape\Neda\Control01\cervical\proj, something is wrong



Cannot check R://Archiv_Tape\Rat_Wistar_5wk\Head_Whole_10um\proj, something is wrong



Cannot check R://Archiv_Tape\Lung Smoke\Lung-Smoke_4mth_003-354_4um\proj, something is wrong


...


Cannot check R://Archiv_Tape\TKI_Skulls\Mouse26563_Skull_Gaby_TKI_7_96um_Al05_2K\proj, something is wrong



Cannot check R://Archiv_Tape\TKI_Skulls\Mouse26567_Skull_Gaby_TKI_7_96um_Al05_2K\proj, something is wrong



Cannot check R://Archiv_Tape\TKI_Skulls\Mouse1268_Skull_Gaby_TKI_7_96um_Al05_2K\proj, something is wrong



Cannot check R://Archiv_Tape\TKI_Skulls\Mouse1269_Skull_Gaby_TKI_7_96um_Al05_2K\proj, something is wrong



Cannot check R://Archiv_Tape\TKI_Skulls\Mouse26564_Skull_Gaby_TKI_7_96um_Al05_2K\proj, something is wrong


In [268]:
Data.sample(n=5)

,LogFile,Folder,LastFolder,NumProjFromLog,NumProjFromDirectory,Machine,SameSame,CheckedTIFFs,CheckedGood,SameSamePlusMinusOne,ScanDate,Difference
2153,R://Archiv_Tape\Eugenio mice\Eugenio mice\c2m9...,R://Archiv_Tape\Eugenio mice\Eugenio mice\c2m9...,proj,1920,1919,Skyscan1172,False,[R://Archiv_Tape\Eugenio mice\Eugenio mice\c2m...,"[True, True, True]",Plus,2018-03-21 08:02:54,1
218,R://Archiv_Tape\Leg-Finland\Leg-Finland\MRb-10...,R://Archiv_Tape\Leg-Finland\Leg-Finland\MRb-10...,proj,769,768,Skyscan1172,False,[R://Archiv_Tape\Leg-Finland\Leg-Finland\MRb-1...,"[True, True]",Plus,2015-09-09 18:05:58,1
1277,R://Archiv_Tape\Validation-Hindlimb\M19\Maus19...,R://Archiv_Tape\Validation-Hindlimb\M19\Maus19...,Proj,1894,1894,SkyScan1272,True,[R://Archiv_Tape\Validation-Hindlimb\M19\Maus1...,"[True, True, True]",Same,2016-02-02 22:56:37,0
1220,R://Archiv_Tape\Validation-Hindlimb\M18\M18-So...,R://Archiv_Tape\Validation-Hindlimb\M18\M18-So...,Proj,769,768,Skyscan1172,False,[R://Archiv_Tape\Validation-Hindlimb\M18\M18-S...,"[True, True]",Plus,2016-06-14 18:45:20,1
1543,R://Archiv_Tape\Liver-Andreas-Kohler\Liver-And...,R://Archiv_Tape\Liver-Andreas-Kohler\Liver-And...,proj,1921,1921,SkyScan1272,True,[R://Archiv_Tape\Liver-Andreas-Kohler\Liver-An...,"[True, True, True]",Same,2016-02-29 16:58:58,0


In [259]:
print('In total we checked %s log files' % len(Data))

In total we checked 3051 log files


In [295]:
Data.groupby(['SameSame','SameSamePlusMinusOne','Machine']).count()

LogFile  Folder  LastFolder  \
SameSame SameSamePlusMinusOne Machine                                    
False                         SkyScan1272       80      80          80   
                              SkyScan1273       32      32          32   
                              SkyScan2211       16      16          16   
                              Skyscan1172      340     340         340   
         Minus                SkyScan1272        5       5           5   
         Plus                 Skyscan1172     1075    1075        1075   
True     Same                 SkyScan1272     1450    1450        1450   
                              SkyScan1273       35      35          35   
                              SkyScan1275        9       9           9   
                              SkyScan2211        1       1           1   
                              SkyScan2214        8       8           8   

                                           NumProjFromLog  \
SameSame SameSamePlusMinusOne Machine                       
False                         SkyScan1272              80   
                              SkyScan1273              32   
                              SkyScan2211              16   
                              Skyscan1172             340   
         Minus                SkyScan1272               5   
         Plus                 Skyscan1172            1075   
True     Same                 SkyScan1272            1450   
                              SkyScan1273              35   
                              SkyScan1275               9   
                              SkyScan2211               1   
                              SkyScan2214               8   

                                           NumProjFromDirectory  CheckedTIFFs  \
SameSame SameSamePlusMinusOne Machine                                           
False                         SkyScan1272                    80            14   
                              SkyScan1273                    32             3   
                              SkyScan2211                    16             0   
                              Skyscan1172                   340            10   
         Minus                SkyScan1272                     5             5   
         Plus                 Skyscan1172                  1075          1073   
True     Same                 SkyScan1272                  1450          1446   
                              SkyScan1273                    35            35   
                              SkyScan1275                     9             9   
                              SkyScan2211                     1             1   
                              SkyScan2214                     8             8   

                                           CheckedGood  ScanDate  Difference  
SameSame SameSamePlusMinusOne Machine                                         
False                         SkyScan1272           14        80          80  
                              SkyScan1273            3        32          32  
                              SkyScan2211            0        16          16  
                              Skyscan1172           10       340         340  
         Minus                SkyScan1272            5         5           5  
         Plus                 Skyscan1172         1073      1075        1075  
True     Same                 SkyScan1272         1446      1450        1450  
                              SkyScan1273           35        35          35  
                              SkyScan1275            9         9           9  
                              SkyScan2211            1         1           1  
                              SkyScan2214            8         8           8

In [297]:
Data.groupby(['SameSame','SameSamePlusMinusOne']).count()

LogFile  Folder  LastFolder  NumProjFromLog  \
SameSame SameSamePlusMinusOne                                                
False                              468     468         468             468   
         Minus                       5       5           5               5   
         Plus                     1075    1075        1075            1075   
True     Same                     1503    1503        1503            1503   

                               NumProjFromDirectory  Machine  CheckedTIFFs  \
SameSame SameSamePlusMinusOne                                                
False                                           468      468            27   
         Minus                                    5        5             5   
         Plus                                  1075     1075          1073   
True     Same                                  1503     1503          1499   

                               CheckedGood  ScanDate  Difference  
SameSame SameSamePlusMinusOne                                     
False                                   27       468         468  
         Minus                           5         5           5  
         Plus                         1073      1075        1075  
True     Same                         1499      1503        1503

In [298]:
# Check if there are any wrong TIFFs
# If there's no output from this cell, then there are no FALSE checked TIFFs :)
for c, row in Data.iterrows():
    if row['CheckedGood']:
        if False in row['CheckedGood']:
            print(row.Folder, row.CheckedGood)

In [575]:
# Output checks
for c, row in Data.iterrows():
    if row.Difference == 1:
        if '1172' in row.Machine:
            #print(row.ScanDate)
            pass
        elif '1272' in row.Machine:
            #print(row.ScanDate)
            pass
        else:
            print(row.LogFile)
    elif row.Difference > 1:
        if row.NumProjFromDirectory == 0:
            pass
            #print('%4s/%s: No projections found: Check %s' % (c, len(Data), row.LogFile))
        else:
            pass
            #print('%4s/%s: Check %s. The Logfile says we should have %s files, we found %s files' % (c, len(Data),
            #                                                                                        row.LogFile,
            #                                                                                         row.NumProjFromLog,
            #                                                                                         row.NumProjFromDirectory))

In [629]:
def doubleckeckdir(logfile, verbose=False):
    """
    See if there's a name mixup, and that's the reason we don't find anything
    Or the logfile is the first of a series of connected scans...
    """
    import re  # for regex searching
    if verbose:
        print(80*'-')
        print('Based on %s' % logfile)
        print('Checking number of TIF files in %s' % os.path.dirname(logfile))
    with open(logfile, 'r') as f:
        prefix = []
        for line in f:
            # Get the prefix of the image files, which is handy for oversize scans
            if 'Filen' in line and 'fix' in line:
                prefix = line.split('=')[1].strip()
    if not prefix:
        print('No "prefix" found in %s' % logfile)
        return()
    with open(logfile, 'r') as f:
        for line in f:
            # Get the prefix of the image files, which is handy for oversize scans
            if 'Number of connected scans' in line:
                connectedscans = int(line.split('=')[1].strip())
    if verbose:
        print('This scan is part of a %s-connected scan' % connectedscans)
    if verbose:
        print('We are only looking for files with %sNUMERALS.tif' % prefix)  
    # Search for all the first tiff files of  tiffiles, but only with with 'prefix' + numerals + .tif, hence exclding prefix_pp*.tif and prefix_ar*.tif
    # regex search based on https://stackoverflow.com/a/55810892/323100 and regexr.com/51lob
    tiffiles = [os.path.join(os.path.dirname(logfile), f) for f in os.listdir(os.path.dirname(logfile)) if re.search(rf'{prefix}([0-9])*\.tif', f)]
    if verbose:
        print('We found %s such files' % len(tiffiles))
    if not len(tiffiles):
        connectedfiles = [f for f in os.listdir(os.path.dirname(logfile)) if re.search(rf'{prefix}~0([0-9])0*\.tif', f)]
        try:
            if len(connectedfiles) == connectedscans:
                return('Connected')
        except:
            return('B0rked')
            print('Something is wrong with %s' % logfile)
    try:
        return(len(tiffiles))
    except:
        return('B0rked')

In [630]:
# Double-check for connected scan
Data['CheckConnected'] = [doubleckeckdir(l) for l in Data['LogFile']]

R://Archiv_Tape\Helsinki\Helsinki\RM91_2_7um_cu\proj\RM91_2_7um_cu.log
R://Archiv_Tape\Heart Finland 2018\Heart Finland 2018\RM91_2_7um_cu\proj\RM91_2_7um_cu.log
R://Archiv_Tape\Immunology\Immunology\BumannA_496444-6AmbHCHB\Probe2-3\proj\BumannA_496444-6AmbHCHB_Probe2_3_.log
R://Archiv_Tape\Immunology\Immunology\BumannA_496444-6AmbHCHB\Probe2-3\proj\BumannA_496444-6AmbHCHB_Probe2_2_.log
R://Archiv_Tape\Immunology\Immunology\BumannA_496444-6AmbHCHB\Probe2\proj\BumannA_496444-6AmbHCHB_Probe2_3_.log
R://Archiv_Tape\Immunology\Immunology\BumannA_496444-6AmbHCHB\Probe2\proj\BumannA_496444-6AmbHCHB_Probe2_Kor_.log
R://Archiv_Tape\Immunology\Immunology\BumannA_496444-6AmbHCHB\Probe2\proj\BumannA_496444-6AmbHCHB_Probe2_2_.log
R://Archiv_Tape\Immunology\Immunology\BumannA_496444-6AmbHCHB\Probe2-2\proj2\BumannA_496444-6AmbHCHB_Probe2_3_.log
R://Archiv_Tape\Immunology\Immunology\BumannA_496444-6AmbHCHB\Probe2-2\proj2\BumannA_496444-6AmbHCHB_Probe2_2_.log
R://Archiv_Tape\Immunology\Immunology\Buma

KeyboardInterrupt: 

In [631]:
Data.sample(n=10)

,LogFile,Folder,LastFolder,NumProjFromLog,NumProjFromDirectory,Machine,SameSame,CheckedTIFFs,CheckedGood,SameSamePlusMinusOne,ScanDate,Difference
72,R://Archiv_Tape\Helsinki\Helsinki\MR3267_2_7um...,R://Archiv_Tape\Helsinki\Helsinki\MR3267_2_7um...,proj,3600,2289,SkyScan1272,False,[R://Archiv_Tape\Helsinki\Helsinki\MR3267_2_7u...,"[True, True, True, True]",,2018-05-27 05:07:27,1311
2304,R://Archiv_Tape\Heart_Finland\Heart_Finland\RK...,R://Archiv_Tape\Heart_Finland\Heart_Finland\RK...,proj,798,797,Skyscan1172,False,[R://Archiv_Tape\Heart_Finland\Heart_Finland\R...,"[True, True]",Plus,2015-02-04 01:59:47,1
226,R://Archiv_Tape\Leg-Finland\Leg-Finland\MRb-11...,R://Archiv_Tape\Leg-Finland\Leg-Finland\MRb-11...,proj,769,768,Skyscan1172,False,[R://Archiv_Tape\Leg-Finland\Leg-Finland\MRb-1...,"[True, True]",Plus,2015-09-14 21:38:57,1
183,R://Archiv_Tape\Schittny\M275\S-E2-LL\02um\pro...,R://Archiv_Tape\Schittny\M275\S-E2-LL\02um\proj,proj,1895,1895,SkyScan1272,True,[R://Archiv_Tape\Schittny\M275\S-E2-LL\02um\pr...,"[True, True, True]",Same,2019-07-13 10:58:54,0
2164,R://Archiv_Tape\Eugenio mice\Eugenio mice\c2m7...,R://Archiv_Tape\Eugenio mice\Eugenio mice\c2m7...,proj,1920,1919,Skyscan1172,False,[R://Archiv_Tape\Eugenio mice\Eugenio mice\c2m...,"[True, True, True]",Plus,2018-03-18 12:32:28,1
3023,R://Archiv_Tape\TKI_Skulls\Mouse5161\overview\...,R://Archiv_Tape\TKI_Skulls\Mouse5161\overview\...,proj,940,940,SkyScan1272,True,[R://Archiv_Tape\TKI_Skulls\Mouse5161\overview...,"[True, True]",Same,2019-10-31 23:39:46,0
953,R://Archiv_Tape\Zebra-Fish_Ines\SCAF-KO-Contro...,R://Archiv_Tape\Zebra-Fish_Ines\SCAF-KO-Contro...,proj,322,322,SkyScan1272,True,[R://Archiv_Tape\Zebra-Fish_Ines\SCAF-KO-Contr...,"[True, True]",Same,2017-10-31 13:19:52,0
2805,R://Archiv_Tape\Rabbit-Grenoble\Rabbit-1-Greno...,R://Archiv_Tape\Rabbit-Grenoble\Rabbit-1-Greno...,proj,583,582,Skyscan1172,False,[R://Archiv_Tape\Rabbit-Grenoble\Rabbit-1-Gren...,"[True, True]",Plus,2015-04-20 13:46:40,1
1061,R://Archiv_Tape\Validation-Hindlimb\M02\M2-Hin...,R://Archiv_Tape\Validation-Hindlimb\M02\M2-Hin...,proj,798,797,Skyscan1172,False,[R://Archiv_Tape\Validation-Hindlimb\M02\M2-Hi...,"[True, True]",Plus,2015-03-20 20:37:09,1
911,R://Archiv_Tape\Zebra-Fish_Ines\SCAF-KC-Contro...,R://Archiv_Tape\Zebra-Fish_Ines\SCAF-KC-Contro...,proj,322,322,SkyScan1272,True,[R://Archiv_Tape\Zebra-Fish_Ines\SCAF-KC-Contr...,"[True, True]",Same,2017-11-06 13:13:43,0


In [35]:
# Save out the dataframe to disk, so we can use it later in another notebook
Data.to_pickle(os.path.join('ConsistencyCheck.pkl'))